In [1]:
import IPython.display
import textwrap
import ipywidgets
import traitlets
import yt

/Users/nathanael/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
ds = yt.load("../IsolatedGalaxy/galaxy0030/galaxy0030")
ds.index

yt : [INFO     ] 2018-05-14 21:04:04,170 Parameters: current_time              = 0.0060000200028298
yt : [INFO     ] 2018-05-14 21:04:04,171 Parameters: domain_dimensions         = [32 32 32]
yt : [INFO     ] 2018-05-14 21:04:04,172 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2018-05-14 21:04:04,173 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2018-05-14 21:04:04,175 Parameters: cosmological_simulation   = 0.0
Parsing Hierarchy : 100%|██████████| 173/173 [00:00<00:00, 19094.62it/s]
yt : [INFO     ] 2018-05-14 21:04:04,201 Gathering a field list (this may take a moment.)


In [30]:
display(ds.fields)

In [90]:
def display_arr(arr):
    unit_registry = arr.units.registry #yt units registry
    dimension = arr.units.dimensions
    equiv = unit_registry.same_dimensions_as(arr.units) #No function 'list_same_dimensions'?
    dropdown = ipywidgets.Dropdown(options = sorted(equiv), value = str(arr.units))
    def arr_updater(arr, texts):
        def _value_updater(change):
            arr2 = arr.in_units(change['new'])
            if arr2.shape == ():
                arr2 = [arr2]
            for v, t in zip(arr2, texts):
                t.value = str(v.value)
        return _value_updater
    if arr.shape == ():
        arr_iter = [arr]
    else:
        arr_iter = arr
    texts = [ipywidgets.Text(value = str(_.value), disabled = True) for _ in arr_iter]
    dropdown.observe(arr_updater(arr, texts), names="value")
    #INFO: Widget properties are IPython traitlets and traitlets are eventful. 
    #      To handle changes, the 'observe' method of the widget can be used to register a callback.
    return ipywidgets.HBox(texts + [dropdown])

In [73]:
print(ds.domain_width)
#print(ds.domain_width.units.registry.to_json())
print(ds.domain_width.units)
print(ds.domain_width.units.dimensions)
print(ds.domain_width.shape)

[1. 1. 1.] code_length
code_length
(length)
(3,)


In [74]:
display_arr(ds.domain_width)

AttributeError: 'UnitRegistry' object has no attribute 'same_dimensions_as'

In [66]:
ds.unit_registry.lut['pc']

(3.0856775809623245e+18, (length), 0.0, '\\rm{pc}')

In [67]:
def display_ds(ds):
    attrs = ["domain_width", "domain_left_edge", "domain_right_edge", "current_time"]
    lb = ipywidgets.VBox([ ipywidgets.Label(value = _) for _ in attrs])
    vb = ipywidgets.VBox([ display_arr(getattr(ds, _)) for _ in attrs])
    hb = ipywidgets.HBox([lb, vb])
    return hb

In [68]:
attrs = ["domain_width", "domain_left_edge", "domain_right_edge", "current_time"]
[getattr(ds, _) for _ in attrs]

[YTArray([1., 1., 1.]) code_length,
 YTArray([0., 0., 0.]) code_length,
 YTArray([1., 1., 1.]) code_length,
 0.0060000200028298 code_time]

In [69]:
display_ds(ds)

TraitError: Invalid selection: value not found

In [ ]:
import ipywidgets as widgets # Loads the Widget framework.
from IPython.core.magics.namespace import NamespaceMagics # Used to query namespace.

# For this example, hide these names, just to avoid polluting the namespace further
get_ipython().user_ns_hidden['widgets'] = widgets
get_ipython().user_ns_hidden['NamespaceMagics'] = NamespaceMagics

In [9]:
class VariableInspectorWindow(object):
    instance = None

    def __init__(self, ipython):
        """Public constructor."""
        if VariableInspectorWindow.instance is not None:
            raise Exception("""Only one instance of the Variable Inspector can exist at a
                time.  Call close() on the active instance before creating a new instance.
                If you have lost the handle to the active instance, you can re-obtain it
                via `VariableInspectorWindow.instance`.""")

        VariableInspectorWindow.instance = self
        self.closed = False
        self.namespace = NamespaceMagics()
        self.namespace.shell = ipython.kernel.shell

        self._box = widgets.Box()
        self._box.layout.overflow_y = 'scroll'
        self._table = widgets.HTML(value = 'Not hooked')
        self._box.children = [self._table]

        self._ipython = ipython
        self._ipython.events.register('post_run_cell', self._fill)

    def close(self):
        """Close and remove hooks."""
        if not self.closed:
            self._ipython.events.unregister('post_run_cell', self._fill)
            self._box.close()
            self.closed = True
            VariableInspectorWindow.instance = None

    def _fill(self):
        """Fill self with variable information."""
        values = self.namespace.who_ls()
        self._table.value = '<div class="rendered_html jp-RenderedHTMLCommon"><table><thead><tr><th>Name</th><th>Type</th><th>Value</th></tr></thead><tr><td>' + \
            '</td></tr><tr><td>'.join(['{0}</td><td>{1}</td><td>{2}'.format(v, type(eval(v)).__name__, str(eval(v))) for v in values]) + \
            '</td></tr></table></div>'

    def _ipython_display_(self):
        """Called when display() or pyout is used to display the Variable
        Inspector."""
        self._box._ipython_display_()


In [10]:
get_ipython()

In [11]:
inspector = VariableInspectorWindow(ds.field_list)

NameError: name 'NamespaceMagics' is not defined

In [75]:
v = 'this'

In [76]:
ds.fields

In [77]:
ds.fields.gas.density.sampling_type

'cell'

In [78]:
density = ds.fields.gas.density

In [79]:
import inspect

In [80]:
inspect.getclosurevars(density._function)

ClosureVars(nonlocals={'field_name': ('enzo', 'Density')}, globals={}, builtins={}, unbound={'copy'})

In [81]:
def display_field_source(fobj):
    source = ipywidgets.Textarea(value = fobj.get_source())
    ac = ipywidgets.Accordion([source])
    ac.set_title(0, "Source")
    return ac

In [82]:
ac = display_field_source(density)

In [83]:
ac.children

(Textarea(value='    def _TranslationFunc(field, data):\n        # We do a bunch of in-place modifications, so we will copy this.\n        return data[field_name].copy()\n'),)

In [84]:
ac.children[0].style.width = '100%'

In [85]:
ac

Accordion(children=(Textarea(value='    def _TranslationFunc(field, data):\n        # We do a bunch of in-plac…

In [86]:
import pygments

In [87]:
ds.fields.field_types

{'all', 'deposit', 'enzo', 'gas', 'index', 'io'}

In [88]:
def display_all_fields(ds):
    fnames = []
    children = []
    for ftype in sorted(ds.fields.field_types):
        fnc = getattr(ds.fields, ftype)
        names = dir(fnc)
        names.sort()
        def change_field(_ftype, _box, _var_window):
            def _change_field(event):
                fobj = getattr(_ftype, event['new'])
                _box.clear_output()
                with _box:
                    IPython.display.display(IPython.display.Markdown(data = "```python\n" + textwrap.dedent(fobj.get_source()) + "\n```"))
                values = inspect.getclosurevars(fobj._function).nonlocals
                _var_window.value = '<div class="rendered_html jp-RenderedHTMLCommon"><table><thead><tr><th>Name</th><th>Type</th><th>Value</th></tr></thead><tr><td>' + \
            '</td></tr><tr><td>'.join(['{0}</td><td>{1}</td><td>{2}'.format(v, type(values[v]).__name__, str(values[v])) for v in sorted(values)]) + \
            '</td></tr></table></div>'
            return _change_field
        flist = ipywidgets.Select(options = names, layout = ipywidgets.Layout(height = '95%'))
        source = ipywidgets.Output(layout = ipywidgets.Layout(width = '100%', height = '9em'))
        var_window = ipywidgets.HTML(value = 'Empty')
        var_box = ipywidgets.Box(layout = ipywidgets.Layout(width = '100%', height = '100%', overflow_y = 'scroll'))
        var_box.children = [var_window]
        ftype_tabs = ipywidgets.Tab(children = [source, var_box], layout = ipywidgets.Layout(flex = '2 1 auto', width = 'auto', height = '95%'))
        ftype_tabs.set_title(0, "Source")
        ftype_tabs.set_title(1, "Variables")
        flist.observe(change_field(fnc, source, var_window), "value")
        children.append(ipywidgets.HBox([flist, ftype_tabs], layout = ipywidgets.Layout(height = '14em')))
        fnames.append(ftype)
    tabs = ipywidgets.Tab(children = children)
    for i, n in enumerate(fnames):
        tabs.set_title(i, n)
    return tabs

In [89]:
display_all_fields(ds)

In [ ]:
from yt.units.unit_registry import UnitRegistry

In [ ]:
reg = UnitRegistry()

In [ ]:
for i in reg.unit_objs: print(i)

In [ ]:
reg['A'][0]

In [ ]:
from yt.units.unit_object import default_unit_registry

In [ ]:
default_unit_registry.unit_objs.items()